# TABLEAU PROJECT

## Geospatial Business Intelligence (BI)


#### El objetivo del proyecto es determinar el punto geográfico donde recomendariamos abrir otra sede de nuestra empresa, teniendo en cuenta los siguientes datos:

   Debemos realizar un análisis geoespacial del dataset "companies"

   Datos sobre nuestra empresa:

   - Es una empresa de software de 50 empleados.

   - La empresa diseña videojuegos.

   - Puestos en nuestra compañía: 20 desarrolladores, 20 Diseñadores/creativos/UX/UI y 10 ejecutivos/managers

   Criterios para la localización de la nueva sede:

   - Debe haber ingenieros de software alrededor

   - Debe haber un buen ratio de grandes compañías vs startups

   - Asegurate de que las compañias cercanas cubren los intereses de tu equipo

   - Evita las viejas empresas, rodeate de empresas nuevas

        

1. Importamos las librerias necesarias para trabajar:

In [1]:
import pymongo
MongoClient = pymongo.MongoClient
import time
client = MongoClient()
import pandas as pd
import re
db = client.companies

In [2]:
new_companies = db.companies.find({'founded_year': { '$gte': 2008} })
len(list(new_companies))

3308

2. Para seleccionar empresas que se dediquen a actividades similares a la nuestra, buscamos todos los "category_code" de las empresas de la BBDD:

In [3]:
cursor2 = db.companies.find({'category_code':{'$exists': 'true'}})
category_codes = list(map(lambda r:r['category_code'], list(cursor2)))

category_code_clean = []
for i in category_codes:
    if i not in category_code_clean:
        category_code_clean.append(i)
    else:
        pass
print(category_code_clean)

['enterprise', 'web', 'software', 'news', 'network_hosting', 'games_video', 'music', 'social', 'mobile', 'search', 'advertising', 'messaging', 'security', 'photo_video', 'finance', 'ecommerce', 'hardware', 'travel', 'public_relations', 'other', 'real_estate', 'semiconductor', 'analytics', 'health', 'legal', 'sports', 'biotech', 'cleantech', 'education', 'consulting', 'transportation', None, 'hospitality', 'fashion', 'nonprofit', 'nanotech', 'automotive', 'design', 'manufacturing', 'government', 'local', 'medical']


3. Vamos a seleccionar las empresas que se dedican a : 'web', 'software', 'network_hosting', 'games_video', 'ecommerce', 'hardware', 'analytics':

In [4]:
#no funciona con "new_companies" en vez de ·companies"

category_companies = db.companies.find({'$and':
                                       [{'founded_year': { '$gte': 2008} }, 
                                         { '$or': 
                                          [{'category_code': 'web'}, 
                                           {'category_code':'software'}, 
                                           {'category_code':'network_hosting'},
                                           {'category_code': 'games_video'},
                                           {'category_code':'ecommerce'},
                                           {'category_code':'hardware'}, 
                                           {'category_code':'network_hosting'},
                                           {'category_code': 'games_video'},
                                           {'category_code':'ecommerce'},
                                           {'category_code':'hardware'}]}]})
len(list(category_companies))

1971

4. Nos quedamos con las empresas que no tengan año de Deadpool:

In [5]:
#cursor = db.companies.find({ '$and': [{'founded_year': { '$gte': 2000} }, {'founded_year': {'$lte': 2005 } } ]})
#print(list(map(lambda r :(r["name"], r['founded_year']), list(cursor))))
category_companies_no_dp = db.companies.find({'$and':
                                       [{'founded_year': { '$gte': 2008} }, 
                                         { '$or': 
                                          [{'category_code': 'web'}, 
                                           {'category_code':'software'}, 
                                           {'}category_code':'network_hosting'},
                                           {'category_code': 'games_video'},
                                           {'category_code':'ecommerce'},
                                           {'category_code':'hardware'}, 
                                           {'category_code':'network_hosting'},
                                           {'category_code': 'games_video'},
                                           {'category_code':'ecommerce'},
                                           {'category_code':'hardware'}]},
                                       {'deadpooled_year': { '$type': 10 }}]})
len(list(category_companies_no_dp))

1824

5. Analizamos los datos del campo "total_money_raised":

In [6]:
cursor3 = db.companies.find({'total_money_raised':{'$exists': 'true'}})
total_money_raised = list(map(lambda r:r['total_money_raised'], list(cursor3)))

total_money_raised_list = []
for i in total_money_raised:
    if i not in total_money_raised_list:
        total_money_raised_list.append(i)
    else:
        pass
print(total_money_raised_list)

['$0', '$39.8M', '$45M', '$800k', '$18.5M', '$18.1M', '$25.8M', '$73.1M', '$2.43B', '$1.16B', '$44.2M', '$16.5M', '$5.58M', '$13.2M', '$28.3M', '$22.5M', '$32.1M', '$106M', '$2.5M', '$6.7M', '$21M', '$73M', '$75.5M', '$69.8M', '$88.7M', '$8.4M', '$1M', '$5.2M', '$18.3M', '$23.4M', '$178M', '$4.7M', '$9M', '$6.8M', '$120M', '$5M', '$33M', '$3M', '$14.7M', '$60.1M', '$24M', '$11M', '$11.5M', '$4M', '$15M', '$76.8M', '$52.5M', '$7M', '$4.99M', '$12.7M', '$3.1M', '$100k', '$32.2M', '$18M', '$53.4M', '$25M', '$34.7M', '$12.5M', '$450k', '$24.3M', '$11.2M', '$50k', '$32M', '$90M', '€225k', '$20.6M', '$68.5M', '$136M', '$555M', '$2M', '$4.1M', '$78M', '$10M', '$8M', '$13.8M', '$26M', '$46M', '$10.5M', '$16M', '$40.4M', '$7.5M', '$1.25M', '$70M', '$1.6M', '$140M', '$200M', '$409M', '$6.5M', '$66M', '$110M', '$14.3M', '$96.8M', '$5.6M', '$31.2M', '$6.2M', '$19.6M', '$23.1M', '$44M', '$22.3M', '$81.3M', '$7.18M', '$8.5M', '$46.8M', '$131M', '$40.1M', '$55.6M', '$52M', '$300k', '$17.5M', '$28.7M'

Eliminamos las que total_money_raised = '$0':

In [7]:
category_companies_no_dp_act = db.companies.find({'$and':
                                       [{'founded_year': { '$gte': 2008} }, 
                                         { '$or': 
                                          [{'category_code': 'web'}, 
                                           {'category_code':'software'}, 
                                           {'category_code':'network_hosting'},
                                           {'category_code': 'games_video'},
                                           {'category_code':'ecommerce'},
                                           {'category_code':'hardware'}, 
                                           {'category_code':'network_hosting'},
                                           {'category_code': 'games_video'},
                                           {'category_code':'ecommerce'},
                                           {'category_code':'hardware'}]},
                                       {'deadpooled_year': { '$type': 10 }},
                                       {'total_money_raised':{'$nin': ['$0']}}]})
x = list(category_companies_no_dp_act)
len(x)

245

In [8]:
x

[{'_id': ObjectId('52cdef7c4bab8bd675297e30'),
  'name': 'GoingOn',
  'permalink': 'goingon',
  'crunchbase_url': 'http://www.crunchbase.com/company/goingon',
  'homepage_url': 'http://www.goingon.com',
  'blog_url': 'http://goingon.com/blog',
  'blog_feed_url': '',
  'twitter_username': 'GoingOn_Network',
  'category_code': 'software',
  'number_of_employees': 40,
  'founded_year': 2008,
  'founded_month': 2,
  'founded_day': 1,
  'deadpooled_year': None,
  'deadpooled_month': None,
  'deadpooled_day': None,
  'deadpooled_url': None,
  'tag_list': 'socialnetworking, enterprise, social-media, social-software',
  'alias_list': None,
  'email_address': 'more@goingon.com',
  'phone_number': '415-200-4606',
  'description': 'Private Academic Networks',
  'created_at': 'Fri Jul 20 04:55:26 UTC 2007',
  'updated_at': 'Thu Apr 04 09:03:17 UTC 2013',
  'overview': '<p>GoingOn provides a fast and cost effective way to build vibrant online communities. Leveraging the best of todayâ€™s social web

6. Del listado de las 245 empresas, creamos la Dataframe: 

In [9]:
df = pd.DataFrame(x)
df.head()

,_id,acquisition,acquisitions,alias_list,blog_feed_url,blog_url,category_code,competitions,created_at,crunchbase_url,...,phone_number,products,providerships,relationships,screenshots,tag_list,total_money_raised,twitter_username,updated_at,video_embeds
0,52cdef7c4bab8bd675297e30,None,[],None,,http://goingon.com/blog,software,"[{'competitor': {'name': 'Ning by Glam Media',...",Fri Jul 20 04:55:26 UTC 2007,http://www.crunchbase.com/company/goingon,...,415-200-4606,"[{'name': 'GoingOn', 'permalink': 'goingon'}, ...",[],"[{'is_past': False, 'title': 'CEO', 'person': ...",[],"socialnetworking, enterprise, social-media, so...",$8.5M,GoingOn_Network,Thu Apr 04 09:03:17 UTC 2013,[]
1,52cdef7c4bab8bd675297f76,None,[],,,,games_video,[],Thu Aug 30 11:29:22 UTC 2007,http://www.crunchbase.com/company/crunchyroll,...,,"[{'name': 'crunchyroll.com', 'permalink': 'cru...",[],"[{'is_past': False, 'title': 'beneficial owner...",[],"crunchyroll, anime, manga, video, gaming, musi...",$4.8M,crunchyroll,Fri Dec 20 11:00:02 UTC 2013,[]
2,52cdef7c4bab8bd675297fec,None,[],,http://blog.muktar.com/feed/,http://blog.muktar.com,web,"[{'competitor': {'name': 'Kluster', 'permalink...",Sat Sep 08 08:24:15 UTC 2007,http://www.crunchbase.com/company/fixya,...,,"[{'name': 'Fixya', 'permalink': 'fixya'}]","[{'title': 'Accounting', 'is_past': False, 'pr...","[{'is_past': False, 'title': 'Founder & CEO', ...",[],"fixya, crowdsourcing, diy, repair, customer-se...",$8M,MUaliyu,Tue Nov 19 03:48:00 UTC 2013,"[{'embed_code': '<div style=""width:320px;""><di..."
3,52cdef7c4bab8bd67529801f,None,[],,,http://corp.wamba.com,web,"[{'competitor': {'name': 'Vostu', 'permalink':...",Fri Sep 28 09:37:07 UTC 2007,http://www.crunchbase.com/company/wamba,...,,"[{'name': 'Wamba', 'permalink': 'wamba'}]","[{'title': 'IT Services Provider', 'is_past': ...","[{'is_past': True, 'title': 'Founder', 'person...",[],socialnetwork,€3M,,Wed May 29 18:58:44 UTC 2013,[]
4,52cdef7c4bab8bd67529803d,"{'price_amount': None, 'price_currency_code': ...","[{'price_amount': None, 'price_currency_code':...",,http://www.uptake.com/blog/feed,http://www.uptake.com/blog/,web,"[{'competitor': {'name': 'TripAdvisor', 'perma...",Thu Oct 11 12:07:18 UTC 2007,http://www.crunchbase.com/company/uptake,...,(650) 329-9888,"[{'name': 'Uptake', 'permalink': 'kango'}]","[{'title': '', 'is_past': False, 'provider': {...","[{'is_past': False, 'title': 'VP of Revenue', ...",[],"travel, search, naturallanguage, semantic",$16.8M,uptake,Mon Dec 23 11:36:17 UTC 2013,[]


6. Eliminamos las filas cuya columna Offices no tiene info:

In [10]:
df['l'] = df.offices.apply(len)

7. Creamos dos columnas ('lat' y 'long'), con los datos que estaban incluidos en la columna 'office':

In [11]:
df1 = df.loc[df['l'] > 0]
df1.reset_index(inplace=True)
df1.offices[0]
df1.offices[0][0]['longitude'] # df.offices -> el primer registro, la primera office, latitude

df1['lat'] = df1.apply(lambda row: row['offices'][0]['latitude'], axis=1)
df1['long'] = df1.apply(lambda row: row['offices'][0]['longitude'], axis=1)
df1.lat.dtype

/home/laura/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/laura/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


dtype('float64')

8. Eliminamos las filas que no tienen datos de 'lat' y 'ĺong':

In [12]:
df1['lat'].isnull().sum()

df2 = df1.dropna(subset=['lat'])
df2.reset_index(inplace=True)
df2.columns

Index(['level_0', 'index', '_id', 'acquisition', 'acquisitions', 'alias_list',
       'blog_feed_url', 'blog_url', 'category_code', 'competitions',
       'created_at', 'crunchbase_url', 'deadpooled_day', 'deadpooled_month',
       'deadpooled_url', 'deadpooled_year', 'description', 'email_address',
       'external_links', 'founded_day', 'founded_month', 'founded_year',
       'funding_rounds', 'homepage_url', 'image', 'investments', 'ipo',
       'milestones', 'name', 'number_of_employees', 'offices', 'overview',
       'partners', 'permalink', 'phone_number', 'products', 'providerships',
       'relationships', 'screenshots', 'tag_list', 'total_money_raised',
       'twitter_username', 'updated_at', 'video_embeds', 'l', 'lat', 'long'],
      dtype='object')

8. Unimos latitud y longitud en una columna, que llamaremos "coord":

In [15]:
df2['coord'] = df2[['lat', 'long']].apply(list, axis=1)
df2.coord

/home/laura/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


0        [37.782263, -122.392142]
1        [37.781265, -122.393229]
2        [37.566879, -122.323895]
3        [37.443058, -122.161674]
4         [40.740231, -73.985433]
5        [34.047312, -118.445243]
6         [38.257035, -85.717393]
7        [37.446823, -122.161523]
8         [40.219214, -74.370313]
9           [52.132633, 5.291266]
10        [40.723652, -73.985281]
11       [37.503147, -122.211435]
12       [37.772323, -122.214897]
13       [34.052187, -118.243425]
14        [34.01052, -118.495981]
15       [37.446382, -122.155678]
16       [34.153588, -118.422495]
17       [37.775196, -122.419204]
18         [40.66509, -73.980306]
19     [36.9751032, -122.0253784]
20     [37.7893215, -122.4013624]
21          [42.35888, -71.05682]
22       [37.269175, -119.306607]
23          [51.51819, -0.208979]
24             [32.0554, 34.7595]
25          [51.125693, 0.257568]
26        [40.757929, -73.985506]
27         [42.275263, -71.24762]
28       [53.544711, -113.515769]
29       [33.5

In [17]:
'''
def nested_to_list(dicts):
    lat_lon = [{"type":"Point","coordinates":[l['long'], l['lat']]} for l in dicts]
    lst = lat_lon[0]
    long = lst['coordinates'][0]
    if long != None:
        return lst
    else:
        return 0
 '''   
def toGeoJSON(array):
    return {
       "type":"Point",
       "coordinates":array}
df2['position'] = df2.apply(lambda r:toGeoJSON(r['coord']),axis=1)
df2.position


/home/laura/.local/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


0      {'type': 'Point', 'coordinates': [37.782263, -...
1      {'type': 'Point', 'coordinates': [37.781265, -...
2      {'type': 'Point', 'coordinates': [37.566879, -...
3      {'type': 'Point', 'coordinates': [37.443058, -...
4      {'type': 'Point', 'coordinates': [40.740231, -...
5      {'type': 'Point', 'coordinates': [34.047312, -...
6      {'type': 'Point', 'coordinates': [38.257035, -...
7      {'type': 'Point', 'coordinates': [37.446823, -...
8      {'type': 'Point', 'coordinates': [40.219214, -...
9      {'type': 'Point', 'coordinates': [52.132633, 5...
10     {'type': 'Point', 'coordinates': [40.723652, -...
11     {'type': 'Point', 'coordinates': [37.503147, -...
12     {'type': 'Point', 'coordinates': [37.772323, -...
13     {'type': 'Point', 'coordinates': [34.052187, -...
14     {'type': 'Point', 'coordinates': [34.01052, -1...
15     {'type': 'Point', 'coordinates': [37.446382, -...
16     {'type': 'Point', 'coordinates': [34.153588, -...
17     {'type': 'Point', 'coord

In [ ]:
8. Seleccionamos las columnas que nos interesan:

In [ ]:
df3=df2[['name','category_code','founded_year','number_of_employees', 'total_money_raised', 'coord']]
df3.head()

In [ ]:
#Creamos las funciones toGeoJSON, que te devuelve, de un  array, el punto y las 2 coordenadas: 
    
#def toGeoJSON(array):
 #   return {
  #      "type":"Point",
   #     "coordinates":[array[0],array[1]]
     #       }

#Creamos la funcion convertRestaurant, que te devuelve el nombre, tipo de cocina, y el resultado de toGeoJSON:

#def convertRestaurant(restaurant):
 #   return {
  #      "name": restaurant["name"],
   #     "cuisine":restaurant["cuisine"],
    #    "position": toGeoJSON(restaurant["address"]["coord"])
    #}

#cursor = db.restaurants.find({"address.coord":{"$size":2}},{"_id":0})
#geoRestaurants = list(map(lambda r: convertRestaurant(r),cursor))

#def convertCompanies(companies):
 #   return {
   #     "name": companies["name"],
  #      "number_of_employees":companies["number_of_employees"],
    #    "category_code": companies["category_code"],
     #   "founded_year":companies["founded_year"],
      #  "offices":toGeoJSON(companies ["coord"])
       #     }
#["offices"]["latitude"]["longitude"]
#display(convertCompanies(df3))


9. Pasamos el data a .JSON

In [ ]:
df3.to_json("companies_with_coords.json", orient="records", lines=True)